<a href="https://colab.research.google.com/github/LSDtopotools/lsdtt_notebooks/blob/master/lsdtopotools/basic_examples/downloading_topographic_data_and_looking_at_it.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downloading topographic data and looking at it

Last updated by Simon M Mudd on 15/12/2023

In this short tutorial you will learn how to download topographic data and look at the hillshade. 
I give a few different examples of how to run the hillshade operation. 

If you use this software for your research, please cite https://doi.org/10.5281/zenodo.3245040

## Stuff we need to do if you are in colab (not required in the lsdtopotools pytools container)

**If you are in the `docker_lsdtt_pytools` docker container, you do not need to do any of this. 
The following is for executing this code in the google colab environment only.**

If you are in the docker container you can skip to the **Download some data** section. 

First we install `lsdtopotools`. The first line downloads the package and the second installs it. The `/dev/null` stuff is just to stop the notebook printing a bunch of text to screen.  

In [ ]:
!wget https://pkgs.geos.ed.ac.uk/geos-jammy/pool/world/l/lsdtopotools2/lsdtopotools2_0.9-1geos~22.04.1_amd64.deb  &> /dev/null
!apt install ./lsdtopotools2_0.9-1geos~22.04.1_amd64.deb  &> /dev/null

The next line tests to see if it worked. If you get some output asking for a parameter file then `lsdtopotools` is installed. This notebook was tested on version 0.9.

In [ ]:
!lsdtt-basic-metrics -v

Now we install `lsdviztools`:

In [ ]:
!pip install lsdviztools  &> /dev/null

## Getting data from OpenTopography.org

OpenTopograhy (https://www.opentopography.org/) is a great place to download topographic data. 

It has vast quantities of lidar data that you can download from the site. In addition there is an API for downloading global datasets (at a range of resolutions). 

`lsdviztools` has a script that can download this global data and convert it to the file format and coordinate reference system (CRS) you need for analysis in `lsdtopotools`. 

To do this you need to have a user account at OpenTopography. You can sign up to an opentopography.org account here: https://portal.opentopography.org/myopentopo 

Once you have done that, you should get your API key and paste it into a text file somewhere safe (if any digital security experts want to explain how to better do this I am all ears). You will need to point to that file when calling the `lsdviztools` opentopography scraper. 

If you are working in *colab* then you should upload the file into your working directory. 

In [ ]:
import lsdviztools.lsdbasemaptools as bmt
from lsdviztools.lsdplottingtools import lsdmap_gdalio as gio

# YOU NEED TO PUT YOUR API KEY IN A FILE
your_OT_api_key_file = "my_OT_api_key.txt"

with open(your_OT_api_key_file, 'r') as file:
    print("I am reading you OT API key from the file "+your_OT_api_key_file)
    api_key = file.read().rstrip()
    print("Your api key starts with: "+api_key[0:4])

Dataset_prefix = "Diablo"
source_name = "COP30"

SB_DEM = bmt.ot_scraper(source = source_name,
                        lower_left_coordinates = [35.1920215020742,-120.90387764783046], 
                        upper_right_coordinates = [35.296562615076155, -120.73735491824398],
                        prefix = Dataset_prefix, 
                        api_key_file = your_OT_api_key_file)
SB_DEM.print_parameters()
SB_DEM.download_pythonic()
DataDirectory = "./"
Fname = Dataset_prefix+"_"+source_name+".tif"
gio.convert4lsdtt(DataDirectory,Fname)

## A simple hillshade image

`lsdtopotools` has various command line functions that run analyses. The one with all the basic topographic analyses is called `lsdtt-basic-metrics`.

We can run `lsdtt-basic-metrics` with a little interface in `lsdviztools` called the `lsdtt_driver`. Here we are only going to write a hillshade raster.  

In [ ]:
import lsdviztools.lsdmapwrappers as lsdmw

In [ ]:
lsdtt_parameters = {"write_hillshade" : "true"}
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
w_prefix = Dataset_prefix+"_"+source_name +"_UTM"
lsdtt_drive = lsdmw.lsdtt_driver(read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

We can plot the image using a `lsdviztools` function. 

In [ ]:
%matplotlib inline
Base_file = r_prefix
this_img = lsdmw.SimpleHillshade(DataDirectory,Base_file,cmap="gist_earth", save_fig=False, size_format="geomorphology",dpi=500)

You can save the figure as well. This allows you to show it in this notebook in a larger format:

In [ ]:
%%capture 
# Use the capture comment to get rid of all the text
# But it will also not display an inline image
# So you need to call the image from the next line of code           
Base_file = r_prefix
this_img = lsdmw.SimpleHillshade(DataDirectory,Base_file,cmap="gist_earth", 
                                 save_fig=True, size_format="geomorphology",dpi=600)

In [ ]:
print(this_img)
from IPython.display import display, Image
display(Image(filename=this_img, width=800))